In [1]:
PATH = "data/"
with open("./major_blogs.txt") as f:
    lines = f.readlines()
len(lines)

30

# Clean data and write to csv

## Process and clean data

In [188]:
import re
from bs4 import BeautifulSoup
import sys
sys.path.append("./")
from utils import *

db = [clean_uni_major_blog(f"{PATH}/{line.strip()}") for line in lines]

In [189]:
def remove_unnecessary_content_in_major(page_content):
    page_content = re.sub(r'\n.+?\|', '', page_content)
    page_content = page_content.replace("Tuyển sinh Nhảy đến nội dung x\n", "")
    page_content = page_content.replace("\nĐăng ký xét tuyển Đặt câu hỏi ", "")
    page_content = re.sub(r'Phương thức tuyển sinh.*?Gọi hotline tư vấn', '', page_content, flags=re.DOTALL)
    page_content = re.sub(r'Vì sao nên chọn ngành.*?Các chương trình của ngành', 'Các chương trình của ngành', page_content, flags=re.DOTALL)
    page_content = re.sub(r'Cựu sinh viên nói gì.*$', '', page_content, flags=re.DOTALL)
    page_content = re.sub(r'Xem thông tin chi tiết về (chương trình đào tạo Chuẩn đầu ra|chuẩn đầu ra Phương thức xét tuyển riêng)', '', page_content)
    page_content = re.sub(r'\n+', '\n', page_content)
    page_content = re.sub(r'\nTìm hiểu về [^\n]+\nWebsite [^\n]+\nKhám phá [^\n]+\n', '\n', page_content)
    return page_content

In [190]:
def extract_major_keywords(page_contents):
    major_keywords = []

    for page_content in page_contents:
        opening_match = re.search(r'\n(.+?) \|', page_content)
        major_keyword = opening_match.group(1) if opening_match else None
        major_keywords.append(major_keyword)

    return major_keywords


def extract_major_info(page_content, major_keyword):
    # Remove unnecessary content
    page_content = remove_unnecessary_content(page_content)

    # Extract major
    major_match = re.search(major_keyword, page_content)
    major = major_match.group(0) if major_match else None

    # Extract description
    description_match = re.search(f'{major_keyword}(.+?)Thông tin cần biết', page_content, re.DOTALL)
    description = description_match.group(1).strip() if description_match else None
    if major and description and major_keyword in description[len(major_keyword):]:
        description = description.replace(major, '', 1).strip()

    # Extract aditional description
    additional_description_match = re.search(r'Thông tin cần biết(.+?)$', page_content, re.DOTALL)
    additional_description = additional_description_match.group(1).replace('Thông tin cần biết ', '').strip() if additional_description_match else None
    additional_description = re.sub(r'năm Chương trình đào tạo', 'năm', additional_description)
    additional_description = re.sub(r'tại Trường liên kết\) Chương trình đào tạo', 'tại Trường liên kết)', additional_description)
    
    return major, description, additional_description



In [191]:
def process_multiple_pages(page_contents, major_keywords):
    data = []

    for i, page_content in enumerate(page_contents):
        major, description, additional_description = extract_major_info(page_content, major_keywords[i])
        data.append({'Major': major, 'Description': description, 'Additional Description': additional_description})

    return data

## write to csv

In [192]:
import csv

major_keywords = extract_major_keywords(db)
result_data = process_multiple_pages(db, major_keywords)

# Create a CSV file and write the extracted information
csv_filename = 'majors_info.csv'

with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Major', 'Description', 'Additional Description']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write data
    for row in result_data:
        writer.writerow(row)

print(f'Data has been written to {csv_filename}.')

Data has been written to majors_info.csv.


# Display data

In [193]:
import pandas as pd

csv_file_path = 'majors_info.csv'

# Read CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the DataFrame
df

,Major,Description,Additional Description
0,Bảo hộ lao động,Bảo hộ lao động là ngành học đón đầu xu hướng ...,Bằng cấp\nCử nhân\nKỹ sư Thời gian đào tạo\nTo...
1,Công nghệ kỹ thuật môi trường - Chuyên ngành C...,Công nghệ kỹ thuật môi trường – Chuyên ngành C...,Bằng cấp\nCử nhân\nKỹ sư Thời gian đào tạo\nTo...
2,Công tác xã hội,Công tác xã hội là ngành khoa học vận dụng nhữ...,Bằng cấp\nCử nhân Thời gian đào tạo\nToàn thời...
3,Kế toán,"Kế toán, kiểm toán không chỉ là công cụ quản l...",Bằng cấp\nCử nhân Thời gian đào tạo\nToàn thời...
4,Khoa học máy tính,Khoa học máy tính là ngành ứng dụng kiến thức ...,Bằng cấp\nCử nhân Thời gian đào tạo\nToàn thời...
5,Khoa học môi trường,"Nền kinh tế của Việt Nam ngày càng phát triển,...",Bằng cấp\nCử nhân\nKỹ sư Thời gian đào tạo\nTo...
6,Kiến trúc,Kiến trúc là ngành tích hợp kiến thức ở cả 2 l...,Bằng cấp\nCử nhân\nKiến trúc sư Thời gian đào ...
7,Kinh doanh quốc tế,Kinh doanh quốc tế đang là một ngành nhận được...,Bằng cấp\nCử nhân Thời gian đào tạo\nToàn thời...
8,Kỹ thuật phần mềm,Kỹ thuật phần mềm là ngành ứng dụng kiến thức ...,Bằng cấp\nCử nhân Thời gian đào tạo\nToàn thời...
9,Kỹ thuật xây dựng công trình giao thông,Kỹ thuật xây dựng công trình giao thông là ngà...,Bằng cấp\nCử nhân\nkỹ sư Thời gian đào tạo\nTo...
